# 2025 Yılı için Enerji Fiyatları - Gelecek Fiyatları

Barış Sanlı, barissanli2@gmail.com


Enerji İstatistik Notu 53'te CME Group'taki gelecek kontratlarının verilerine erişim için kullanılan kod.
Kodun içerisinde "k" değişkenindeki tarihi çalıştırılan tarihten bir gün önceki tarihi kullanmak gerekir

In [1]:
#!pip install fake-useragent

In [2]:
%pylab inline
import requests
from numpy import array
import datetime  
import pandas as pd
from fake_useragent import UserAgent

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
ua = UserAgent()
print(ua.chrome)

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36


In [4]:
an="https://www.cmegroup.com/CmeWS/mvc/Settlements/Futures/Settlements/"
#buradaki tarih olan tradeDate'i değiştirin, son 5 gün fiyatlarını alabilirsiniz. 
k="/FUT?strategy=DEFAULT&tradeDate=12/23/2024&pageSize=500&isProtected&_t=1735137018838"
hh="444"
ttf="8378"
nbp="8376"
jkm="7049"
coal="5951"
brent="424"
tl="8582"
co2="10459"
debaseload="8381"

In [5]:
headers = {'User-Agent':str(ua.chrome)}

In [6]:
cl=an+brent+k
r = requests.get(cl,headers=headers).json()
tx=pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"HH"])
tx

,Date,HH
0,FEB 25,72.63
1,MAR 25,72.32
2,APR 25,71.99
3,MAY 25,71.70
4,JUN 25,71.44
...,...,...
94,DEC 32,66.88
95,JAN 33,66.88
96,FEB 33,66.88
97,MAR 33,66.88


In [7]:
r["dsHeader"]

'Brent Last Day Financial Future'

In [8]:
#HenryHub
r = requests.get(an+hh+k,headers=headers).json()
hh_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"HH"])
#TTF 
r = requests.get(an+ttf+k,headers=headers).json()
ttf_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"TTF"])
#JKM 
r = requests.get(an+jkm+k,headers=headers).json()
jkm_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"JKM"])
#Coal
r = requests.get(an+coal+k,headers=headers).json()
coal_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"CoalAPI2"])
#Brent
r = requests.get(an+brent+k,headers=headers).json()
brent_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"Brent"])
#DE BaseLoad
r = requests.get(an+debaseload+k,headers=headers).json()
debaseload_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"DE_BaseLoad"])
# CO2 - Europe
r = requests.get(an+co2+k,headers=headers).json()
co2_price = pd.DataFrame([(item['month'], item['settle']) for item in r['settlements']],columns=['Date',"CO2"])
# TRY
r = requests.get(an+tl+k,headers=headers).json()
tl_price = [(item['month'], item['settle']) for item in r['settlements']]
tlf=pd.DataFrame(tl_price,columns=['Date',"TRYUSD"])

In [9]:
dfs = [hh_price, ttf_price, jkm_price, coal_price, brent_price, debaseload_price, co2_price, tlf]
dfs = [df.set_index('Date') for df in dfs]
dfm = pd.concat(dfs, axis=1).reset_index()

In [10]:
dfm.head(12)

,Date,HH,TTF,JKM,CoalAPI2,Brent,DE_BaseLoad,CO2,TRYUSD
0,JAN 25,3.65600,45.545,NaN,108.90,NaN,114.88,67.90,NaN
1,FEB 25,3.34600,45.764,13.985,107.70,72.63,108.83,68.08,NaN
2,MAR 25,2.94400,45.667,14.005,107.45,72.32,92.75,68.29,.0260850
3,APR 25,2.93400,45.470,13.895,107.20,71.99,80.90,68.29,NaN
4,MAY 25,3.02600,45.285,13.860,106.95,71.70,72.08,NaN,NaN
5,JUN 25,3.22200,45.212,13.900,107.70,71.44,80.03,68.73,.0239900
6,JLY 25,3.40800,44.970,13.850,108.50,71.20,89.50,NaN,NaN
7,AUG 25,3.45000,44.799,13.950,109.30,70.96,87.85,69.07,NaN
8,SEP 25,3.43400,44.920,14.065,110.15,70.72,94.06,69.25,.0221350
9,OCT 25,3.51200,43.738,13.885,111.00,70.49,98.14,69.25,NaN


In [11]:
dfm.TRYUSD=dfm.TRYUSD[pd.to_numeric(dfm['TRYUSD'], errors='coerce').notnull()]
dfm.TRYUSD=dfm.TRYUSD.astype(float16)
dfm["USDTRY"]=1/dfm.TRYUSD

In [12]:
dfm.USDTRY=dfm.USDTRY.interpolate()
#dfm.USDTRY=dfm.USDTRY.interpolate()
#dfm.CO2=dfm.CO2.interpolate()

In [13]:
dfm.head(12)[["Date","HH","TTF","JKM","CoalAPI2","Brent","DE_BaseLoad","CO2","USDTRY"]]

,Date,HH,TTF,JKM,CoalAPI2,Brent,DE_BaseLoad,CO2,USDTRY
0,JAN 25,3.65600,45.545,NaN,108.90,NaN,114.88,67.90,NaN
1,FEB 25,3.34600,45.764,13.985,107.70,72.63,108.83,68.08,NaN
2,MAR 25,2.94400,45.667,14.005,107.45,72.32,92.75,68.29,38.31250
3,APR 25,2.93400,45.470,13.895,107.20,71.99,80.90,68.29,39.43750
4,MAY 25,3.02600,45.285,13.860,106.95,71.70,72.08,NaN,40.56250
5,JUN 25,3.22200,45.212,13.900,107.70,71.44,80.03,68.73,41.68750
6,JLY 25,3.40800,44.970,13.850,108.50,71.20,89.50,NaN,42.84375
7,AUG 25,3.45000,44.799,13.950,109.30,70.96,87.85,69.07,44.00000
8,SEP 25,3.43400,44.920,14.065,110.15,70.72,94.06,69.25,45.15625
9,OCT 25,3.51200,43.738,13.885,111.00,70.49,98.14,69.25,46.28125
